In [1]:
import pandas as pd
import re 
import nltk
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import spacy
import time
import math
import string
import gensim
import pyLDAvis
import seaborn as sns 
import matplotlib.pyplot as plt
import pyLDAvis.gensim
#import pyLDAvis.gensim_models as gensimvis

from textblob import TextBlob
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from gensim import corpora, models
from gensim.models.ldamulticore import LdaMulticore

In [2]:
import pandarallel
import multiprocessing
from pandarallel import pandarallel

num_processors = multiprocessing.cpu_count()

pandarallel.initialize(nb_workers=num_processors, use_memory_fs=False)

INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [3]:
%%time 

input_path = r"C:\Users\verma\OneDrive - The University of Chicago\Desktop\MSCA\QUARTER 4 SUMMER\NLP\FINAL PROJECT\data.parquet"

df = pd.read_parquet(input_path, engine='pyarrow')

CPU times: total: 23.7 s
Wall time: 1min 26s


In [64]:
df.shape

(199208, 9)

In [65]:
# Calculate the number of rows for 10% of the data
sample = 3000

# Randomly select 10% of the data
sampled_df = df.sample(n=sample, random_state=42)  # Use a fixed seed for reproducibility

In [66]:
sampled_df.shape

(3000, 9)

In [67]:
sampled_df.tail(5)

,date,title,text,year,month,day,dayofweek,clean_title,clean_text
89382,2022-04-13,Razer & Lambda Tensorbook create laptop dedica...,\n\nRazer & Lambda Tensorbook create laptop de...,2022,4,13,2,Razer Lambda Tensorbook create laptop dedicate...,Razer Lambda Tensorbook create laptop dedicate...
126030,2022-01-28,Here's How AI Will Help Nissan Build Even Bett...,Here's How AI Will Help Nissan Build Even Bett...,2022,1,28,4,Heres How AI Will Help Nissan Build Even Bette...,Heres How AI Will Help Nissan Build Even Bette...
190687,2023-03-22,Mozilla launches a new startup focused on ‘tru...,\n\nMozilla launches a new startup focused on ...,2023,3,22,2,Mozilla launches a new startup focused on trus...,Mozilla launches a new startup focused on trus...
176174,2023-06-21,"Europe, US urged to investigate the type of AI...","\n\n \n\n \n\nEurope, US urged to investigat...",2023,6,21,2,"Europe, US urged to investigate the type of AI...","Europe, US urged to investigate the type of A..."
104124,2023-05-19,"Sean Penn, backing WGA strike, says AI dispute...","\n\n \n\n \nSean Penn, backing WGA strike, s...",2023,5,19,4,"Sean Penn, backing WGA strike, says AI dispute...","Sean Penn, backing WGA strike, says AI disput..."


### Tokenization 

In [68]:
lemma = WordNetLemmatizer()

In [69]:
stopwords = set(nltk.corpus.stopwords.words('english'))
exclude = set(string.punctuation)

In [70]:
def tokens(df,column):
    import nltk
    lemma = WordNetLemmatizer()
    stopwords = set(nltk.corpus.stopwords.words('english'))
    exclude = set(string.punctuation)
    
    new_col = column + "_tokens"
    sampled_df[new_col] = sampled_df[column].parallel_apply(lambda x: nltk.tokenize.word_tokenize(x))
    
    sampled_df[new_col] = sampled_df[new_col].parallel_apply(lambda x: [lemma.lemmatize(word) for word in x])
    sampled_df[new_col] = sampled_df[new_col].parallel_apply(lambda x: [word.lower() for word in x if word.isalpha()])
    sampled_df[new_col] = sampled_df[new_col].parallel_apply(lambda x: [word for word in x if word not in stopwords])
    sampled_df[new_col] = sampled_df[new_col].parallel_apply(lambda x: [word for word in x if len(word)>1])
    sampled_df[new_col] = sampled_df[new_col].parallel_apply(lambda x: list((x)))

In [71]:
%time tokens(sampled_df, "clean_title")

CPU times: total: 438 ms
Wall time: 13.3 s


In [72]:
sampled_df['article_length'] = sampled_df['clean_text'].apply(len)

In [73]:
sampled_df['title_length'] = sampled_df['clean_title'].apply(len)

In [74]:
sampled_df.head(5)

,date,title,text,year,month,day,dayofweek,clean_title,clean_text,clean_title_tokens,article_length,title_length
135881,2022-06-08,QYSEA's Innovative AI Vision Lock Platform Set...,QYSEA's Innovative AI Vision Lock Platform Set...,2022,6,8,2,QYSEAs Innovative AI Vision Lock Platform Sets...,QYSEAs Innovative AI Vision Lock Platform Sets...,"[qyseas, innovative, ai, vision, lock, platfor...",7989,95
87337,2023-05-25,PayPal Is Using AI to Supercharge Profits | Na...,\nPayPal Is Using AI to Supercharge Profits | ...,2023,5,25,3,PayPal Is Using AI to Supercharge Profits Nasdaq,PayPal Is Using AI to Supercharge Profits Nasd...,"[paypal, using, ai, supercharge, profits, nasdaq]",7090,48
117475,2023-02-12,Professor Cath Ellis: ChatGPT And The Controve...,\n \nProfessor Cath Ellis: ChatGPT And The Con...,2023,2,12,6,Professor Cath Ellis: ChatGPT And The Controve...,Professor Cath Ellis: ChatGPT And The Controv...,"[professor, cath, ellis, chatgpt, controversy,...",12694,94
54075,2023-02-24,Facebook Parent Meta Wants to Show It's Still ...,\n\n\nFacebook Parent Meta Wants to Show It's ...,2023,2,24,4,Facebook Parent Meta Wants to Show Its Still a...,Facebook Parent Meta Wants to Show Its Still a...,"[facebook, parent, meta, wants, show, still, b...",4689,76
66928,2021-03-23,Artificial Intelligence Market is Expected to ...,\nArtificial Intelligence Market is Expected t...,2021,3,23,1,Artificial Intelligence Market is Expected to ...,Artificial Intelligence Market is Expected to ...,"[artificial, intelligence, market, expected, g...",6953,122


In [75]:
# Calculate the 55th percentile length
percentile_55 = np.percentile(sampled_df['article_length'], 55)

print("55th Percentile Length:", percentile_55)

55th Percentile Length: 7469.9


In [76]:
# Filter articles based on length (keeping articles shorter than or equal to the 55th percentile length)
filtered_sample_df = sampled_df[sampled_df['article_length'] <= percentile_55]

# Print the filtered DataFrame
filtered_sample_df.head(5)

,date,title,text,year,month,day,dayofweek,clean_title,clean_text,clean_title_tokens,article_length,title_length
87337,2023-05-25,PayPal Is Using AI to Supercharge Profits | Na...,\nPayPal Is Using AI to Supercharge Profits | ...,2023,5,25,3,PayPal Is Using AI to Supercharge Profits Nasdaq,PayPal Is Using AI to Supercharge Profits Nasd...,"[paypal, using, ai, supercharge, profits, nasdaq]",7090,48
54075,2023-02-24,Facebook Parent Meta Wants to Show It's Still ...,\n\n\nFacebook Parent Meta Wants to Show It's ...,2023,2,24,4,Facebook Parent Meta Wants to Show Its Still a...,Facebook Parent Meta Wants to Show Its Still a...,"[facebook, parent, meta, wants, show, still, b...",4689,76
66928,2021-03-23,Artificial Intelligence Market is Expected to ...,\nArtificial Intelligence Market is Expected t...,2021,3,23,1,Artificial Intelligence Market is Expected to ...,Artificial Intelligence Market is Expected to ...,"[artificial, intelligence, market, expected, g...",6953,122
25858,2022-07-23,Google fires employee who said its conversatio...,Google fires employee who said its conversatio...,2022,7,23,5,Google fires employee who said its conversatio...,Google fires employee who said its conversatio...,"[google, fire, employee, said, conversation, a...",5754,62
182624,2021-06-22,Radically oversimplified AI skin-tone tool cou...,\n \n\nRadically oversimplified AI skin-tone t...,2021,6,22,1,Radically oversimplified AI skintone tool coul...,Radically oversimplified AI skintone tool cou...,"[radically, oversimplified, ai, skintone, tool...",6406,81


In [77]:
filtered_sample_df.shape

(1650, 12)

In [78]:
filtered_sample_df.head()

,date,title,text,year,month,day,dayofweek,clean_title,clean_text,clean_title_tokens,article_length,title_length
87337,2023-05-25,PayPal Is Using AI to Supercharge Profits | Na...,\nPayPal Is Using AI to Supercharge Profits | ...,2023,5,25,3,PayPal Is Using AI to Supercharge Profits Nasdaq,PayPal Is Using AI to Supercharge Profits Nasd...,"[paypal, using, ai, supercharge, profits, nasdaq]",7090,48
54075,2023-02-24,Facebook Parent Meta Wants to Show It's Still ...,\n\n\nFacebook Parent Meta Wants to Show It's ...,2023,2,24,4,Facebook Parent Meta Wants to Show Its Still a...,Facebook Parent Meta Wants to Show Its Still a...,"[facebook, parent, meta, wants, show, still, b...",4689,76
66928,2021-03-23,Artificial Intelligence Market is Expected to ...,\nArtificial Intelligence Market is Expected t...,2021,3,23,1,Artificial Intelligence Market is Expected to ...,Artificial Intelligence Market is Expected to ...,"[artificial, intelligence, market, expected, g...",6953,122
25858,2022-07-23,Google fires employee who said its conversatio...,Google fires employee who said its conversatio...,2022,7,23,5,Google fires employee who said its conversatio...,Google fires employee who said its conversatio...,"[google, fire, employee, said, conversation, a...",5754,62
182624,2021-06-22,Radically oversimplified AI skin-tone tool cou...,\n \n\nRadically oversimplified AI skin-tone t...,2021,6,22,1,Radically oversimplified AI skintone tool coul...,Radically oversimplified AI skintone tool cou...,"[radically, oversimplified, ai, skintone, tool...",6406,81


In [80]:
def process_text(text):
    from nltk.tokenize import word_tokenize
    tokens = word_tokenize(text)
    tokens = [lemma.lemmatize(word) for word in tokens]
    tokens = [word.lower() for word in tokens if word.isalpha()]
    tokens = [word for word in tokens if word not in stopwords]
    # You can add more preprocessing steps here
    return tokens

In [81]:
%time filtered_sample_df['clean_text_tokens'] = filtered_sample_df['clean_text'].apply(process_text)

CPU times: total: 5.28 s
Wall time: 7.41 s


<timed exec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [84]:
filtered_sample_df.head(5)

,date,title,text,year,month,day,dayofweek,clean_title,clean_text,clean_title_tokens,article_length,title_length,clean_text_tokens
87337,2023-05-25,PayPal Is Using AI to Supercharge Profits | Na...,\nPayPal Is Using AI to Supercharge Profits | ...,2023,5,25,3,PayPal Is Using AI to Supercharge Profits Nasdaq,PayPal Is Using AI to Supercharge Profits Nasd...,"[paypal, using, ai, supercharge, profits, nasdaq]",7090,48,"[paypal, using, ai, supercharge, profits, nasd..."
54075,2023-02-24,Facebook Parent Meta Wants to Show It's Still ...,\n\n\nFacebook Parent Meta Wants to Show It's ...,2023,2,24,4,Facebook Parent Meta Wants to Show Its Still a...,Facebook Parent Meta Wants to Show Its Still a...,"[facebook, parent, meta, wants, show, still, b...",4689,76,"[facebook, parent, meta, wants, show, still, b..."
66928,2021-03-23,Artificial Intelligence Market is Expected to ...,\nArtificial Intelligence Market is Expected t...,2021,3,23,1,Artificial Intelligence Market is Expected to ...,Artificial Intelligence Market is Expected to ...,"[artificial, intelligence, market, expected, g...",6953,122,"[artificial, intelligence, market, expected, g..."
25858,2022-07-23,Google fires employee who said its conversatio...,Google fires employee who said its conversatio...,2022,7,23,5,Google fires employee who said its conversatio...,Google fires employee who said its conversatio...,"[google, fire, employee, said, conversation, a...",5754,62,"[google, fire, employee, said, conversation, a..."
182624,2021-06-22,Radically oversimplified AI skin-tone tool cou...,\n \n\nRadically oversimplified AI skin-tone t...,2021,6,22,1,Radically oversimplified AI skintone tool coul...,Radically oversimplified AI skintone tool cou...,"[radically, oversimplified, ai, skintone, tool...",6406,81,"[radically, oversimplified, ai, skintone, tool..."


In [82]:
filtered_sample_df.shape

(1650, 13)

In [83]:
output_path = r"C:\Users\verma\OneDrive - The University of Chicago\Desktop\MSCA\QUARTER 4 SUMMER\NLP\FINAL PROJECT/filter_data.parquet"
filtered_sample_df.to_parquet(output_path, index=False)